In [1]:
import sys
import pandas as pd
import configparser
from io import StringIO , BytesIO

sys.path.append('/home/renan/dev/python/project-tcc/dags/')
config = configparser.ConfigParser()
config.read('../notebooks/config-dev.ini')

['../notebooks/config-dev.ini']

In [2]:
from util import datalake
from util.data import creater_analysis, pre_processing

data_lake_user = config['azure']['data_lake_user']
data_lake_key = config['azure']['data_lake_key']
fs_name = config['datalake']['fs_name']

In [4]:
client = datalake.DataLake(data_lake_user, data_lake_key, fs_name)
client.list_files('raw_data')

['raw_data/2014',
 'raw_data/2014/Benefits_Cost_Sharing_PUF.csv',
 'raw_data/2014/Business_Rules_PUF.csv',
 'raw_data/2014/Network_PUF.csv',
 'raw_data/2014/Plan_Attributes_PUF_2014_2015-03-09.csv',
 'raw_data/2014/Rate_PUF.csv',
 'raw_data/2014/Service_Area_PUF.csv',
 'raw_data/2015',
 'raw_data/2015/Benefits_Cost_Sharing_PUF.csv',
 'raw_data/2015/Business_Rules_PUF_Reformat.csv',
 'raw_data/2015/Network_PUF.csv',
 'raw_data/2015/Plan_Attributes_PUF.csv',
 'raw_data/2015/Rate_PUF.csv',
 'raw_data/2015/Service_Area_PUF.csv',
 'raw_data/2016',
 'raw_data/2016/Benefits_Cost_Sharing_PUF_2015-12-08.csv',
 'raw_data/2016/Business_Rules_PUF_2015-12-08.csv',
 'raw_data/2016/Network_PUF_2015-12-08.csv',
 'raw_data/2016/Plan_Attributes_PUF_2015-12-08.csv',
 'raw_data/2016/Rate_PUF_2015-12-08.csv',
 'raw_data/2016/ServiceArea_PUF_2015-12-08.csv',
 'raw_data/data',
 'raw_data/data/Plan_Crosswalk_PUF_2014-12-22.csv',
 'raw_data/data/Plan_ID_Crosswalk_PUF_2015-12-07.CSV']

In [5]:
buffer, _, metadata = client.get_file('raw_data/2016/', 'Network_PUF_2015-12-08.csv')
df = pd.read_csv(BytesIO(buffer))
metadata.metadata

{'content_hash': '2016_Network_PUF_2015-12-08.csv',
 'filename': 'Network_PUF_2015-12-08.csv',
 'id': '9',
 'schema': 'Network',
 'year': '2016'}

In [4]:
df = pd.read_csv('../data/PlanAttributes.csv', encoding="latin1", low_memory=False)

In [6]:
df.head(2)

,BusinessYear,StateCode,IssuerId,SourceName,VersionNum,ImportDate,IssuerId2,StateCode2,NetworkName,NetworkId,NetworkURL,RowNumber,MarketCoverage,DentalOnlyPlan
0,2016,AK,21989,HIOS,4,2015-08-22 15:09:32,21989,AK,Delta Dental Premier,AKN001,https://www.modahealth.com/ProviderSearch/face...,13,NaN,NaN
1,2016,AK,21989,HIOS,4,2015-08-22 15:09:32,21989,AK,Delta Dental PPO,AKN002,https://www.modahealth.com/ProviderSearch/face...,14,NaN,NaN


In [7]:
df.shape

(1426, 14)

In [8]:
df_analysis = creater_analysis.create_schema_analysis(df)
display(df_analysis.head(40))

,column_name,column_name_formatted,percent_filling,column_type_dataframe,column_type,column_size,column_percent_unique
0,BusinessYear,BUSINESSYEAR,100.00,int64,INTEGER,None,0.07
1,StateCode,STATECODE,100.00,object,VARCHAR,2,2.66
2,IssuerId,ISSUERID,100.00,int64,INTEGER,None,54.00
3,SourceName,SOURCENAME,100.00,object,VARCHAR,5,0.21
4,VersionNum,VERSIONNUM,100.00,int64,INTEGER,None,1.33
5,ImportDate,IMPORTDATE,100.00,object,VARCHAR,19,5.96
6,IssuerId2,ISSUERID2,100.00,int64,INTEGER,None,54.00
7,StateCode2,STATECODE2,100.00,object,VARCHAR,2,2.66
8,NetworkName,NETWORKNAME,100.00,object,VARCHAR,100,44.18
9,NetworkId,NETWORKID,100.00,object,VARCHAR,6,16.27


In [10]:
def verifica_colunas_unicas(df):
    colunas_unicas = []
    for coluna in df.columns:
        if df[coluna].nunique() == len(df):
            colunas_unicas.append(coluna)
    return colunas_unicas

colunas_unicas = verifica_colunas_unicas(df)

# Exibindo as colunas únicas encontradas
if len(colunas_unicas) > 0:
    print("As seguintes colunas são únicas:")
    for coluna in colunas_unicas:
        print(coluna)
else:
    print("Não há colunas únicas no DataFrame.")

Não há colunas únicas no DataFrame.


In [11]:
# adding primary key
df_analysis.insert(1,"is_primary_key", False)
df_analysis.insert(1,"is_anonymizated_field", False)

In [87]:
if 1 > 10:
    # setting primary key
    df_analysis['is_primary_key'].mask(df_analysis["column_name_formatted"] == "NUMERO_DO_SUBCONTRATO", True, inplace=True)

    # setting anonimyzated column    
    df_analysis['is_anonymizated_field'].mask(df_analysis["column_name_formatted"] == "NOME_CLIENTE", True, inplace=True)


In [13]:
df_analysis = df_analysis.add_prefix('metadata.metadata['schema'] '+ "_")
display(df_analysis.head(5))

,PlanAttributes_column_name,PlanAttributes_is_anonymizated_field,PlanAttributes_is_primary_key,PlanAttributes_column_name_formatted,PlanAttributes_percent_filling,PlanAttributes_column_type_dataframe,PlanAttributes_column_type,PlanAttributes_column_size,PlanAttributes_column_percent_unique
0,AVCalculatorOutputNumber,False,False,AVCALCULATOROUTPUTNUMBER,70.49,object,VARCHAR,18,10.24
1,BeginPrimaryCareCostSharingAfterNumberOfVisits,False,False,BEGINPRIMARYCARECOSTSHARINGAFTERNUMBEROFVISITS,100.00,int64,INTEGER,None,0.01
2,BeginPrimaryCareDeductibleCoinsuranceAfterNumb...,False,False,BEGINPRIMARYCAREDEDUCTIBLECOINSURANCEAFTERNUMB...,100.00,int64,INTEGER,None,0.01
3,BenefitPackageId,False,False,BENEFITPACKAGEID,100.00,int64,INTEGER,None,0.09
4,BusinessYear,False,False,BUSINESSYEAR,100.00,int64,INTEGER,None,0.00


In [15]:
new_name_file = 'metadata.metadata['schema']'  + "_validator.json"

with open("../data/json/" + new_name_file, 'w', encoding='utf-8') as file:
    df_analysis.to_json(file, force_ascii=False)

In [16]:
df_validator = pd.read_json("../data/json/" + new_name_file) 
display(df_validator.head(15))

,PlanAttributes_column_name,PlanAttributes_is_anonymizated_field,PlanAttributes_is_primary_key,PlanAttributes_column_name_formatted,PlanAttributes_percent_filling,PlanAttributes_column_type_dataframe,PlanAttributes_column_type,PlanAttributes_column_size,PlanAttributes_column_percent_unique
0,AVCalculatorOutputNumber,False,False,AVCALCULATOROUTPUTNUMBER,70.49,object,VARCHAR,18.0,10.24
1,BeginPrimaryCareCostSharingAfterNumberOfVisits,False,False,BEGINPRIMARYCARECOSTSHARINGAFTERNUMBEROFVISITS,100.00,int64,INTEGER,NaN,0.01
2,BeginPrimaryCareDeductibleCoinsuranceAfterNumb...,False,False,BEGINPRIMARYCAREDEDUCTIBLECOINSURANCEAFTERNUMB...,100.00,int64,INTEGER,NaN,0.01
3,BenefitPackageId,False,False,BENEFITPACKAGEID,100.00,int64,INTEGER,NaN,0.09
4,BusinessYear,False,False,BUSINESSYEAR,100.00,int64,INTEGER,NaN,0.00
5,CSRVariationType,False,False,CSRVARIATIONTYPE,100.00,object,VARCHAR,39.0,0.03
6,ChildOnlyOffering,False,False,CHILDONLYOFFERING,100.00,object,VARCHAR,27.0,0.01
7,ChildOnlyPlanId,False,False,CHILDONLYPLANID,0.56,object,VARCHAR,14.0,0.09
8,CompositeRatingOffered,False,False,COMPOSITERATINGOFFERED,35.40,object,VARCHAR,3.0,0.00
9,DEHBCombInnOonFamilyMOOP,False,False,DEHBCOMBINNOONFAMILYMOOP,1.20,object,VARCHAR,14.0,0.01
